# env

this notebook can only run at the same level as `src`

In [2]:
%load_ext autoreload
%autoreload 2


import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load default config

this is the config data the package take to build graph/model and do train/inference, can be modified

In [3]:
from src.configs.defaults import get_cfg_defaults
cfg = get_cfg_defaults()
print(cfg)

DATASET:
  DATA_PATH: /home/NOBACKUP/mzhou3/599/ns_music_all_data.p
  INTERACTION_DF: df_playlist
  ITEM: track
  ITEM_DF: df_track
  ITEM_FEATURES: [['key', 'CAT'], ['tempo_5cat', 'CAT'], ['livness_5cat', 'CAT'], ['instrumentalness_3cat', 'CAT'], ['speechiness_10cat', 'CAT'], ['loudness_10cat', 'CAT'], ['acousticness_10cat', 'CAT'], ['artist_id', 'CAT'], ['album_id', 'CAT'], ['music_continous_features', 'VEC'], ['genre_old_vec', 'VEC'], ['genres_vec', 'VEC']]
  ITEM_ID: tid
  ITEM_USER_EDGE: contained_by
  NAME: SPOTIFY_MUSIC
  SAMPLER:
    NEIGHBOR_SAMPLER:
      DEFAULT_SAMPLER:
        NUM_LAYERS: 2
        NUM_NEIGHBORS: 3
        NUM_RANDOM_WALKS: 10
        RANDOM_WALK_LENGTH: 2
        RANDOM_WALK_RESTART_PROB: 0.5
      NAME: DEFAULT
    NODES_SAMPLER:
      BATCH_SIZE: 32
      NAME: DEFAULT
      PATH: 
  USER: playlist
  USER_DF: df_playlist_info
  USER_ID: pid
  USER_ITEM_EDGE: contains
FP16: False
MODEL:
  ARCH: PINSAGE
  PINSAGE:
    HIDDEN_SIZE: 128
    LAYERS: 2
    PR

# update config

In [8]:
# change data path 
cfg_data = cfg.DATASET
cfg_data.DATA_PATH = 'ns_music_all_data_ming_20211124'
# specify output path
cfg.OUTPUT_PATH = '/home/NOBACKUP/mzhou3/599/test_run_8'


model changes
- use layer norm at projection and representation
- change representation embedding size to be 64

In [9]:
cfg.MODEL.PINSAGE.PROJECTION.NORMALIZE= True
cfg.MODEL.PINSAGE.REPRESENTATION_NORMALIZE= True
cfg.MODEL.PINSAGE.HIDDEN_SIZE = 64
cfg.MODEL.PINSAGE.PROJECTION.EMB = [['id', 64],
 ['album_id', 64],
 ['artist_id', 64],
 ['key', 16],
 ['tempo_5cat', 8],
 ['livness_5cat', 8],
 ['instrumentalness_3cat', 4],
 ['speechiness_10cat', 16],
 ['loudness_10cat', 16],
 ['acousticness_10cat', 16]]


# build model

In [10]:
from src.model import build_model
from src.data.dataset import build_dataset

In [11]:
g, train_g, [train_user_ids, val_user_ids, test_user_ids] = build_dataset(cfg)
model = build_model(g, cfg)

# load weights

In [14]:
import torch 
model_state = torch.load('weights_fullmodel_size64_epoch5.p', map_location='cpu')

In [15]:
model.load_state_dict(model_state['model_state'])

<All keys matched successfully>

# build sampler

In [17]:
from torch.utils.data import IterableDataset, DataLoader
from src.data.sampler.graph_sampler import build_graph_sampler


neighbor_sampler, collator = build_graph_sampler(train_g, cfg)

dataloader_test = DataLoader(
        torch.arange(g.number_of_nodes('track')),
        batch_size=32,
        collate_fn=collator.collate_test,
        num_workers=4)

# generate embedding

In [18]:
model = model.eval()
model = model.cuda()
device = torch.device('cuda:0')


In [ ]:

all_features_cpu = []
dataloader_it = iter(dataloader_test)
idx =0 
for blocks in dataloader_it:
    idx +=1 
    if idx % 100 ==0:
        print(idx)
    with torch.no_grad():
        for i in range(len(blocks)):
            blocks[i] = blocks[i].to(device)

        features = model.get_repr(blocks)
        all_features_cpu.append(features.cpu().numpy())
all_features_cpu = np.concatenate(all_features_cpu)

# save
#import pickle
#pickle.dump(all_features_cpu, open('embs', 'wb'))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850